In [1]:
script_version='2.6'
date='March 8th, 2018, UNB'

In [2]:
import numpy
import sys

In [3]:
from ReadBatchZipLM_Omit_Accel_Readdata_reg import Y, batch_read_thread, reset, method, filename, omit, reg, input_file_variable_name, max

Opened input file and ready for reading from FallDataAvgFixed32AutoShift_50Miss_2018-03-10  15-17-39.csv
Read output samples: 126 samples from FallDataAvgFixed32AutoShift_50Miss_2018-03-10  15-17-39.csv
Y length:
126


In [4]:
#USE CPU ONLY
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"#'-1'

In [5]:
from sklearn.metrics import average_precision_score, roc_auc_score, roc_curve, f1_score, classification_report, recall_score, brier_score_loss, precision_score, regression
    
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

from keras import metrics
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.callbacks import TensorBoard
from keras.preprocessing import sequence
import tensorflow
import threading
import queue
import pickle
import jsonpickle
import logging
import matplotlib.pyplot as plt


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
class Epoch(object):
    def __init__(self, e, **kwargs):
        self.number=e
        self.data=dict()
        self.input_output=dict()

In [7]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [8]:
model = Sequential()

model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2, input_shape=(32,4), name="L1_LSTM"))

if(not reg):
    if(not omit):
        output_layer=1
        model.add(Dense(output_layer, activation='sigmoid', name="Output_Dense"))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])   
    else:
        #output_layer=4
        output_layer=1
        model.add(Dense(output_layer, activation='sigmoid', name="Output_Dense"))
        model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
else:
    output_layer=1
    model.add(Dense(output_layer, name="Output_Dense_reg"))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
print((model.summary()))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
L1_LSTM (LSTM)               (None, 32)                4736      
_________________________________________________________________
Output_Dense_reg (Dense)     (None, 1)                 33        
Total params: 4,769
Trainable params: 4,769
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
n_epochs = 10

#fill it according to dataset file
n_samples = 453

if(not reg):
    totalSamples,minibatchSize,testPercentage, epochs, minibatch_epochs=(n_samples,n_samples,0.1,n_epochs,10)
else:
    totalSamples,minibatchSize,testPercentage, epochs, minibatch_epochs=(n_samples,n_samples,0.2,n_epochs,10)
    
no_mini_batches=int(totalSamples/minibatchSize)
X_test_all=[]
y_test_all=[]

In [10]:
#print(totalSamples)

In [11]:
import datetime
dateformatFile='%Y-%m-%d  %H-%M-%S'

In [12]:
directory="Results "+str(datetime.datetime.now().strftime(dateformatFile))+' Samples-{0}_MB-{1}_TestPerc{2}_{3}'.format(totalSamples,minibatchSize, int(testPercentage*100),method)
print(directory)
if not os.path.exists(directory):
    os.makedirs(directory)

Results 2018-03-12  21-06-41 Samples-126_MB-126_TestPerc20_reg_Fixed32AutoShift


In [13]:
#filename= FallDataAvgLifeModel32Input.csv

#An empty file with the name of the input file!
#mehrdad(why commented?)
#open(directory+'/'+filename.format("Input"),'wb').close()
logFile=open(directory+'/'+'log.txt','wt')

In [14]:
logging.basicConfig(filename=directory+'/'+'log.txt',format='%(message)s', level=logging.DEBUG)
#Adding log to console as well
consoleHandler=logging.StreamHandler()
consoleHandler.setFormatter(logging.Formatter('*\t%(message)s'))
logging.getLogger().addHandler(consoleHandler)
dateformat="%Y/%m/%d  %H:%M:%S"
logging.info("Script version: "+script_version)
logging.info("Log file created at "+datetime.datetime.now().strftime("%Y/%m/%d  %H:%M:%S"))
logging.info("Directory: {0}".format(directory))
logging.info("Filename Template: {0}".format(filename))
logging.info("Filename Input: {0}".format(filename.format(input_file_variable_name)))
logging.info("Filename Output: {0}".format(filename.format(input_file_variable_name + "Output")))

*	Script version: 2.6
*	Log file created at 2018/03/12  21:06:41
*	Directory: Results 2018-03-12  21-06-41 Samples-126_MB-126_TestPerc20_reg_Fixed32AutoShift
*	Filename Template: FallDataAvgFixed32AutoShift{0}_2018-03-10  15-17-39.csv
*	Filename Input: FallDataAvgFixed32AutoShift_50Miss_2018-03-10  15-17-39.csv
*	Filename Output: FallDataAvgFixed32AutoShift_50MissOutput_2018-03-10  15-17-39.csv


In [15]:
tensorboard = TensorBoard(log_dir=directory, write_graph=True)

In [16]:
startTime=datetime.datetime.now()
logging.info('Start time: '+str(startTime))    
logging.info((model.summary()))
accuracy=[]

metricsReports=[]
q=queue.Queue(1)
epochsData=[]

*	Start time: 2018-03-12 21:06:47.025518
*	None


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
L1_LSTM (LSTM)               (None, 32)                4736      
_________________________________________________________________
Output_Dense_reg (Dense)     (None, 1)                 33        
Total params: 4,769
Trainable params: 4,769
Non-trainable params: 0
_________________________________________________________________


In [17]:
print(Y, batch_read_thread, reset, method, filename.format("Input"), omit,sep='\n')

[0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 0, 1, 2, 3, 4, 0, 1, 2, 3, 0, 1, 2, 3, 4, 5, 0, 1, 2, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 0, 1, 2, 3, 4, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 0, 1, 2, 3, 0, 1, 2, 0, 1, 2, 3, 0, 1, 2, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 3, 0, 1, 2, 0, 1, 2, 3, 4, 0, 1, 2, 0, 1, 2, 3, 4, 0, 1, 2, 3]
<function batch_read_thread at 0x00000246B39BF268>
<function reset at 0x00000246B39BF1E0>
reg_Fixed32AutoShift
FallDataAvgFixed32AutoShiftInput_2018-03-10  15-17-39.csv
False


In [18]:
for e in range(epochs):

    epoch_start_time=datetime.datetime.now()
    logging.info("\t##########################################################################")
    logging.info("\t############# Epoch {0} started @ {1} ####################".format(e+1,str(epoch_start_time.strftime(dateformat))))
    logging.info("\t##########################################################################")
    
    epoch=Epoch(e)
    
    X_test_all=[]
    y_test_all=[]
    X2=[] 
    if e==0:#No need for reseting the file if it is not the first minibatch
        #print("Reading Thread Starting (Epoch Thread)")
        reset()
        readingThread=threading.Thread(target=batch_read_thread,args=(q,minibatchSize))
        readingThread.start()
        
    if e>0:#Should not be necessary anymore
        readingThread.join()
    
    batch_data=[]
    epoch.input_output['all_batches_training_history_loss']=[]
    epoch.input_output['all_batches_training_history_accuracy']=[]
    epoch.input_output['all_batches_training_history_mse']=[]
    
    for minibatch in range(no_mini_batches):
        #Read samples and prepare data
        minibatch_start_time=datetime.datetime.now()
        logging.info("*********Epoch {2} of {3}, Minibatch {0} of {1} @ {4}********".format(minibatch+1,no_mini_batches,e+1,epochs,str(minibatch_start_time.strftime(dateformat))))
        logging.info("Samples:{0}, {3}, MBatch: {1}, Test {2}, Total {4}, MB:{5}".format(totalSamples,minibatchSize,testPercentage,method, int(testPercentage*totalSamples),int(testPercentage*minibatchSize)))
      
        #readingThread.start()        
        readingThread.join()        
        #print("Finished!")                
        X2=q.get()
        #print('Len X2:')
        #print(len(X2))
        #len(X2)
        X=X2
        
        print("X=X2")
        print(len(X2))
        
        te=list(numpy.random.randint(0,len(X),size=int((len(X)*testPercentage))))#Rand 10 percent for testing
        tr=set(list(range(0,len(X))))-set(te)#The remaining 90 percent for training
        
        offset=minibatchSize*minibatch
        
        X_train=[X[i] for i in tr]
        y_train=[Y[offset+i] for i in tr]
        X_test=[X[i] for i in te]
        y_test=[Y[offset+i] for i in te]
        X_test_all.extend(X_test)
        y_test_all.extend(y_test)
        
        if(minibatch==(no_mini_batches-1)):#If last minibatch, reset to read the first batch for next epoch
            reset()

        readingThread=threading.Thread(target=batch_read_thread,args=(q,minibatchSize))
        readingThread.start()
        
        history=model.fit(numpy.array(X_train), numpy.array(y_train),
                          epochs=minibatch_epochs, batch_size=1,
                          callbacks=[tensorboard])
        logging.info("Batch History: ")
        logging.info(str(history.history.items()))
        
        scores = model.evaluate(numpy.array(X_test), numpy.array(y_test))#, verbose=0)
        epoch.input_output['all_batches_training_history_loss'].append((history.history['loss']))
        if(not reg):
            epoch.input_output['all_batches_training_history_accuracy'].append(history.history['acc'])
            logging.info("Test Accuracy: %.3f%%" % (scores[1]*100))
            
        else:
            epoch.input_output['all_batches_training_history_accuracy'].append(history.history['mean_squared_error'])
            logging.info("Test MSE: %.3f%%" % (scores[1]*100))
    
        
        logging.info("**END****Epoch {2} of {3}, Minibatch {0} of {1} in {4} ********".format(minibatch+1,no_mini_batches,e+1,epochs,str((datetime.datetime.now()-minibatch_start_time))))
        logging.info('')
        
    # Final evaluation of the model    
    logging.info("For {0} test samples".format(len(y_test_all)))
    logging.info("True Values")
    logging.info(y_test_all)
    logging.info("Predicted Values")
    result=model.predict_on_batch(numpy.array(X_test_all))
    logging.info(model.predict_on_batch(numpy.array(X_test_all)))
    logging.info("Comparison on all:")
    result_both=[(float(result[i]),round(float(result[i])), y_test_all[i]) for i in range(len(result))]
    if(not omit):
        y_score=[round(float(y)) for y in result]
    else:
        y_score=result
            
    scores = model.evaluate(numpy.array(X_test_all), numpy.array(y_test_all), verbose=0)
    accuracy.append((scores[1]*100, scores[0]))
    
    epoch.data['accuracy']=scores[1]
    epoch.data['loss']=scores[0]
        
    if(not omit):
        if(not reg):
            epoch.data['average_precision']=average_precision_score(y_test_all, y_score)
            #Mehrdad. I commented the line below because when all of 7 test samples are 1, it causes an error.
            #https://stackoverflow.com/questions/39018097/sklearn-auc-valueerror-only-one-class-present-in-y-true
            #epoch.data['roc_auc']=roc_auc_score(y_test_all, y_score)
            epoch.input_output['roc_curve']=roc_curve(y_test_all, y_score)
            epoch.data['f1']=f1_score(y_test_all, y_score)
            epoch.data['recall']=recall_score(y_test_all, y_score)
            epoch.data['precision']=precision_score(y_test_all, y_score)
        
        if(not reg):
            #Calculate Brier Score
            if hasattr(model, "predict_proba"):
                prob_pos = model.predict_proba(numpy.array(X_test_all))#[:, 1]
            else:  # use decision function
                prob_pos = model.decision_function(numpy.array(X_test_all))
                prob_pos = \
                        (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())

            epoch.input_output['prob_pos']=prob_pos

            epoch.input_output['Y_test_all']=y_test_all
            epoch.input_output['y_predictions']=y_score

            clf_score = brier_score_loss(y_test_all, prob_pos, pos_label=1)
            brier=clf_score 

            epoch.data['brier']=clf_score

            logging.info('Brier: {0:0.2f}'.format(clf_score))

    

        #print(':')
        epoch.input_output['classification_report']=classification_report(y_test_all, y_score)
        metricsReports.append(epoch.input_output['classification_report'])
        
    epochsData.append(epoch)
    

    logging.info(result_both)
    logging.info("Predictions on all:")
    
    logging.info("")
    logging.info("*****************************************")
    logging.info("***Epoch Summary***")
    logging.info("Parameters: Samples:{0}, Minibatch Size: {1}, Test ratio: {2} ".format(totalSamples,minibatchSize,testPercentage))    
    logging.info(str(epoch.data))
    logging.info("Loss: %.2f%%" % (scores[0]))
    
    if(not omit):
        if(not reg):
            logging.info('Average precision-recall: {0:0.2f}'.format(epoch.data['average_precision']))
        #mehrdad
        #logging.info('AU-ROC: {0:0.2f}'.format(epoch.data['roc_auc']))   
            logging.info('F1: {0:0.2f}'.format(epoch.data['f1']))    
            logging.info('Recall: {0:0.2f}'.format(epoch.data['recall']))
            logging.info('Precision: {0:0.2f}'.format(epoch.data['precision']))
            logging.info("Accuracy: %.2f%%" % (scores[1]*100))    
            logging.info("Brier: %.2f%%" % (brier))    
            logging.info("*****************************************")
            logging.info("***Classification Reports***")
    
    if(not reg):
        for ep in range(e+1):
            logging.info("Epoch {0} accuracy:{1}, loss:{2}".format(ep+1,accuracy[ep][0],accuracy[ep][1]))
            if(not omit):
                logging.info(metricsReports[ep])
    
    #print("***Metrics***")
    logging.info("***Metrics***")
    for ep in range(e+1):
        if (not reg):
            logging.info("Epoch {0} accuracy:{1}, loss:{2}".format(ep+1,accuracy[ep][0],accuracy[ep][1]))
        else:
            logging.info("Epoch {0} MSE:{1}, loss:{2}".format(ep+1,accuracy[ep][0],accuracy[ep][1]))
        
        #print("Epoch {0} metrics".format(ep+1))        
        logging.info(epochsData[ep].data)

        #print([str(met) for met in metricsAll[e]])
    logging.info("***Results***")
    
    if(not reg):
        for ep in range(e+1):
            logging.info("Epoch {0} Accuracy:{1}, Loss:{2}".format(ep+1,accuracy[ep][0],accuracy[ep][1]))
    else:
        for ep in range(e+1):
            logging.info("Epoch {0} MSE:{1}, Loss:{2}".format(ep+1,accuracy[ep][0],accuracy[ep][1]))
        
    logging.info("Parameters: Samples:{0}, Method={3}, Minibatch Size: {1}, Test: Ratio: {2}, Total {4}, MB:{5}".format(totalSamples,minibatchSize,testPercentage, method, int(testPercentage*totalSamples),int(testPercentage*minibatchSize)))
    if(not reg):
        logging.info("Accuracy: %.2f%%" % (scores[1]*100))
    else:
        logging.info("MSE: %.2f%%" % (scores[1]*100))
    
    logging.info("***END OF EPOCH {0} of {1}** @ {2} in {3}*".format(e+1, epochs,datetime.datetime.now().strftime(dateformat),str((datetime.datetime.now()-epoch_start_time))))
    logging.info("*****************************************")
    logging.info("")
    
    acc_train=[epoch_iter.input_output['all_batches_training_history_accuracy'][-1][-1] for epoch_iter in epochsData]
    loss_train=[epoch_iter.input_output['all_batches_training_history_loss'][-1][-1] for epoch_iter in epochsData]

    acc=[epoch_iter.data['accuracy'] for epoch_iter in epochsData]
    loss=[epoch_iter.data['loss'] for epoch_iter in epochsData]

    plt.plot(acc_train)
    plt.plot(acc)
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')

    plt.tight_layout()
    plt.savefig(directory+'/Accuracy'+str(e+1)+'.pdf',bbox_inches='tight', pad_inches=0)
    #plt.show()
    plt.close()
    
    # summarize history for loss
    plt.plot(loss_train)
    plt.plot(loss)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    #plt.show()
    plt.tight_layout()
    plt.savefig(directory+'/Loss'+str(e+1)+'.pdf',bbox_inches='tight', pad_inches=0)
    plt.close()
    
    if(not reg):
        #Plot Brier
        if(not omit):
            fig = plt.figure(1, figsize=(10, 10))
            ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
            ax2 = plt.subplot2grid((3, 1), (2, 0))

            ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
            fraction_of_positives, mean_predicted_value = \
                    calibration_curve(y_test_all, prob_pos, n_bins=10)

            epoch.input_output['fraction_of_positives']=fraction_of_positives
            epoch.input_output['mean_predicted_value']=mean_predicted_value 


            ax1.plot(mean_predicted_value, fraction_of_positives, "s-",
                         label="%s (%1.3f)" % ('Model', clf_score))

            ax2.hist(prob_pos, range=(0, 1), bins=10, label='LSTM',
                         histtype="step", lw=2)

            ax1.set_ylabel("Fraction of positives")
            ax1.set_xlabel("Mean predicted value")

            ax1.set_ylim([-0.05, 1.05])
            ax1.legend(loc="lower right")
            ax1.set_title('Calibration plots  (reliability curve)')

            ax2.set_xlabel("Mean predicted value")
            ax2.set_ylabel("Count")
            ax2.legend(loc="upper center", ncol=2)

            plt.tight_layout()
            plt.savefig(directory+'/Calibration Epoch '+str(e+1)+'.pdf',bbox_inches='tight', pad_inches=0)
            plt.close()

            #plt.show()

            #Plot Brier for all

                #Plot Brier


            fig = plt.figure(1, figsize=(10, 10))
            ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
            ax2 = plt.subplot2grid((3, 1), (2, 0))

            ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
            fraction_of_positives, mean_predicted_value = \
                    calibration_curve(y_test_all, prob_pos, n_bins=10)

            epoch.input_output['fraction_of_positives']=fraction_of_positives
            epoch.input_output['mean_predicted_value']=mean_predicted_value 


            for x in epochsData:
                ax1.plot(x.input_output['mean_predicted_value'], x.input_output['fraction_of_positives'], "s-",
                        label="%s (%1.3f)" % ('Epoch{0}'.format(str(x.number+1)), x.data['brier']))

            #ax1.plot(mean_predicted_value, fraction_of_positives, "s-",
             #            label="%s (%1.3f)" % ('LSTM', clf_score))

            ax2.hist(prob_pos, range=(0, 1), bins=10, label='LSTM',
                         histtype="step", lw=2)

            ax1.set_ylabel("Fraction of positives")
            ax1.set_xlabel("Mean predicted value")

            ax1.set_ylim([-0.05, 1.05])
            ax1.legend(loc="lower right")
            ax1.set_title('Calibration plots  (reliability curve)')

            ax2.set_xlabel("Mean predicted value")
            ax2.set_ylabel("Count")
            ax2.legend(loc="upper center", ncol=2)

            plt.tight_layout()
            plt.savefig(directory+'/Calibration Epoch All '+str(e+1)+'.pdf',bbox_inches='tight', pad_inches=0)
            plt.close()


            #Area Under the Curve
            fpr, tpr, _= epoch.input_output['roc_curve']

            #mehrdad
            '''
            plt.figure()
            lw = 2
            #plt.plot(fpr, tpr, color='darkorange',
            #     lw=lw, label='ROC curve (area = %0.2f)' % epoch.data['roc_auc'])
            plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver operating characteristic')
            plt.legend(loc="lower right")
            plt.savefig(directory+'/AUC Epoch'+str(e+1)+'.pdf',bbox_inches='tight', pad_inches=0)
            plt.close()
            '''
            #mehrdad
            '''
            plt.figure()
            lw = 2
            for x in epochsData:        
                fpr, tpr, _= x.input_output['roc_curve']
                plt.plot(fpr, tpr,
                     lw=lw, label='ROC curve Epoch{0} (area = %0.2f)'.format(str(x.number+1)) % x.data['roc_auc'])
            plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver operating characteristic')
            plt.legend(loc="lower right")
            plt.savefig(directory+'/AUC Epochs All'+str(e+1)+'.pdf',bbox_inches='tight', pad_inches=0)
            plt.close()
            '''
        
    pickleFile=open(directory+'/'+'data{0}.bin'.format(e+1),'wb')
    jsonFile=open(directory+'/'+'data{0}.txt'.format(e+1),'wt')
    epochFile=open(directory+'/'+'epoch{0}.txt'.format(e+1),'wt')
    
    model.save(directory+'/'+'model_keras{0}.h5'.format(e+1))


    import json
    pickle.dump(epoch,pickleFile)
    pickleFile.flush()
    pickleFile.close()

    epochFile.write(str(epoch.number))
    epochFile.write(str(epoch.data))
    epochFile.write(str(epoch.input_output))
    epochFile.close()

    jsonFile.write(jsonpickle.encode(epoch))        
    #json.dump([epoch.number,epoch.data, epoch.input_output],jsonFile)
    jsonFile.flush()
    jsonFile.close()

*		##########################################################################
*		############# Epoch 1 started @ 2018/03/12  21:06:51 ####################
*		##########################################################################


*	*********Epoch 1 of 10, Minibatch 1 of 1 @ 2018/03/12  21:06:51********


*	Samples:126, reg_Fixed32AutoShift, MBatch: 126, Test 0.2, Total 25, MB:25


**Finished Reading 126 from 0 to 126 Samples***
X=X2
0


ValueError: low >= high

In [ ]:
numpy.array(X).shape

### 

In [ ]:
numpy.array(X)[0][29]

In [ ]:
print(history.history.keys())